In [ ]:
import gpxpy
import pandas as pd
import os
from datetime import timedelta
from typing import List, Tuple
from gpxpy.gpx import GPXTrackPoint
import math

# =========================
# 輔助計算函式
# =========================


def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    R = 6371000  # 地球半徑（公尺）
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)

    a = (
        math.sin(d_phi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c


def calculate_slope(elevation_diff: float, distance: float) -> float:
    if distance == 0:
        return 0.0
    return math.degrees(math.atan(elevation_diff / distance))


# =========================
# 檔案讀取與處理
# =========================


def load_communication_points(filepath: str) -> pd.DataFrame:
    with open(filepath, "r", encoding="utf-8-sig") as f:
        lines = [line.strip() for line in f if line.strip()]

    from io import StringIO

    clean_data = StringIO("\n".join(lines))
    df = pd.read_csv(clean_data, sep="\t")

    df["緯度"] = df["緯度"].str.replace("°", "").astype(float)
    df["經度"] = df["經度"].str.replace("°", "").astype(float)
    df["海拔（約）"] = df["海拔（約）"].str.replace("m", "").str.strip().astype(int)
    df["是否為切點"] = df["是否為切點"].str.strip().str.upper() == "Y"

    return df


def load_gpx_points(filepath: str) -> List[GPXTrackPoint]:
    with open(filepath, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    points = []
    for track in gpx.tracks:
        for segment in track.segments:
            points.extend(segment.points)
    return points


# =========================
# 切點處理與分段
# =========================


def identify_cut_points(df: pd.DataFrame) -> List[Tuple[float, float]]:
    cut_df = df[df["是否為切點"]]
    return list(zip(cut_df["緯度"], cut_df["經度"]))


def segment_by_cutpoints(
    points: List[GPXTrackPoint],
    cut_coords: List[Tuple[float, float]],
    tolerance: float = 0.0002,
) -> List[List[GPXTrackPoint]]:
    segments = []
    current_segment = []
    cut_idx = 0
    for pt in points:
        current_segment.append(pt)
        if cut_idx >= len(cut_coords):
            continue
        lat, lon = cut_coords[cut_idx]
        if abs(pt.latitude - lat) < tolerance and abs(pt.longitude - lon) < tolerance:
            if cut_idx > 0:
                segments.append(current_segment[:-1])
                current_segment = [pt]
            cut_idx += 1

    segments.append(current_segment)

    if cut_idx < len(cut_coords):
        print(
            f"⚠️ 警告：僅匹配到 {cut_idx}/{len(cut_coords)} 個切點，請檢查座標精度或容差設定。"
        )

    return segments


# =========================
# 指標計算與報告生成
# =========================


def analyze_segment(segment: List[GPXTrackPoint], comm_points: pd.DataFrame) -> str:
    valid_segment = [pt for pt in segment if pt.time is not None]
    if len(valid_segment) < 2:
        return "資料不足，無法產生分析報告。"

    segment = valid_segment
    start_time, end_time = segment[0].time, segment[-1].time
    duration = end_time - start_time
    hours, remainder = divmod(duration.total_seconds(), 3600)
    minutes = remainder // 60
    total_time_str = f"{int(hours)}小時{int(minutes)}分鐘"

    elevations = [pt.elevation for pt in segment]
    max_ele, min_ele = max(elevations), min(elevations)
    altitude_range = max_ele - min_ele
    high_altitude_warning = "是" if max_ele > 2438 else "否"

    slopes = []
    max_slope, max_slope_time_diff = 0.0, 0.0
    for i in range(1, len(segment)):
        dist = haversine(
            segment[i - 1].latitude,
            segment[i - 1].longitude,
            segment[i].latitude,
            segment[i].longitude,
        )
        elev_diff = segment[i].elevation - segment[i - 1].elevation
        slope = calculate_slope(elev_diff, dist)
        slopes.append(slope)
        if abs(slope) > abs(max_slope):
            max_slope = slope
            max_slope_time_diff = (
                segment[i].time - segment[i - 1].time
            ).total_seconds() / 60

    slope_series = pd.Series(slopes)
    slope_std_dev = slope_series.std()
    slope_variance = slope_std_dev**2

    freq_dist = (
        pd.cut(slopes, bins=[-90, -10, -5, 0, 5, 10, 90]).value_counts().to_dict()
    )

    lat_range = [pt.latitude for pt in segment]
    lon_range = [pt.longitude for pt in segment]
    mid_comm = comm_points[
        (comm_points["緯度"] >= min(lat_range))
        & (comm_points["緯度"] <= max(lat_range))
        & (comm_points["經度"] >= min(lon_range))
        & (comm_points["經度"] <= max(lon_range))
        & (~comm_points["是否為切點"])
    ]

    # 組合報告
    report = f"統計數據摘要\n"
    report += f"- 總耗時：{total_time_str}\n"
    report += (
        f"- 海拔範圍：{min_ele:.1f}m - {max_ele:.1f}m（範圍：{altitude_range:.1f}m）\n"
    )
    report += f"- 高海拔警示：{high_altitude_warning}\n"
    report += f"- 最大坡度：{max_slope:.2f}°\n"
    report += f"- 坡度標準差：{slope_std_dev:.2f}，變異數：{slope_variance:.2f}\n"
    report += f"- 最大坡度時間差：{max_slope_time_diff:.1f} 分鐘\n"
    report += f"- 中繼點數量：{len(mid_comm)}\n\n"

    report += "定性觀察與詮釋\n"
    report += "- 坡度頻率分布：\n"
    for k, v in freq_dist.items():
        report += f"  {k}: {v} 次\n"
    report += "- 效率分類：依坡度與移動速度綜合評估（尚未實作）\n\n"

    report += "指標定義與分類說明\n"
    report += "- 坡度標準差：地形穩定性指標，變異數＝標準差平方\n"
    report += "- 最大坡度：急陡地形代表，風險指標\n"

    return report


# =========================
# 主程式與執行入口
# =========================


def process_gpx_and_generate_reports(gpx_path: str, comm_txt: str) -> None:
    comm_df = load_communication_points(comm_txt)
    cut_points = identify_cut_points(comm_df)
    points = load_gpx_points(gpx_path)
    segments = segment_by_cutpoints(points, cut_points)

    base_name = os.path.splitext(os.path.basename(gpx_path))[0]
    for idx, seg in enumerate(segments, 1):
        report = analyze_segment(seg, comm_df)
        out_name = f"{base_name}_第{idx}段.txt"
        with open(out_name, "w", encoding="utf-8") as f:
            f.write(f"【{base_name} 第{idx}段報告】\n\n")
            f.write(report)

    print(f"✅ 分析完成，報告已匯出，共 {len(segments)} 段。")


# ========== 執行入口 ==========
if __name__ == "__main__":
    process_gpx_and_generate_reports("「桃山」單攻.gpx", "communication_point.txt")

⚠️ 警告：僅匹配到 1/4 個切點，請檢查座標精度或容差設定。
✅ 分析完成，報告已匯出，共 1 段。


In [ ]:
import gpxpy
import gpxpy.gpx
import pandas as pd
import numpy as np
from haversine import haversine, Unit
from datetime import timedelta
import io
import os

# --- 使用者設定 ---
# 將您的 GPX 檔案路徑放在這裡
GPX_FILE_PATH = "標準桃山.gpx"

# 通訊點資料 (直接內嵌於程式碼中)
COMMUNICATION_POINTS_DATA = """
步道名稱	路標指示	緯度	經度	海拔（約）	是否為切點
桃山瀑布步道	0 k 起點	24.39700°	121.30770°	1400 m	Y
桃山步道	0K_瀑布1K交會	24.40522°	121.30758°	2100 m	Y
桃山步道	1 K	24.41011°	121.31125°	2300 m	N
桃山步道	1.5 K	24.41304°	121.30947°	2500 m	N
桃山步道	2 K	24.41630°	121.30720°	2700 m	Y
桃山步道	3.5 K	24.42640°	121.30377°	3000 m	N
桃山步道	4 K	24.42911°	121.30409°	3050 m	N
桃山步道	4.5 K	24.43251°	121.30463°	3323 m	Y
"""

# --- 核心功能函式 ---


def load_comm_points(data):
    """
    從字串資料中讀取通訊點，並整理成 DataFrame。
    """
    df = pd.read_csv(io.StringIO(data), sep="\t")
    # 清理緯度和經度欄位中的特殊字元
    df["緯度"] = df["緯度"].str.replace("°", "").astype(float)
    df["經度"] = df["經度"].str.replace("°", "").astype(float)
    return df


def parse_gpx(file_path):
    """
    解析 GPX 檔案，將軌跡點轉換為 DataFrame。
    """
    points = []
    with open(file_path, "r", encoding="utf-8") as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append(
                        {
                            "time": point.time,
                            "latitude": point.latitude,
                            "longitude": point.longitude,
                            "elevation": point.elevation,
                        }
                    )
    df = pd.DataFrame(points)
    # 將時間轉換為可操作的格式
    df["time"] = pd.to_datetime(df["time"], utc=True)
    return df


def find_closest_point_index(gpx_df, point, start_search_idx=0):
    """
    在 GPX DataFrame 中，從指定的索引開始，尋找距離給定點最近的點的索引。
    """
    min_dist = float("inf")
    closest_idx = -1

    # 建立一個 NumPy 陣列以進行向量化計算
    gpx_points = gpx_df[["latitude", "longitude"]].iloc[start_search_idx:].to_numpy()
    target_point = (point["緯度"], point["經度"])

    # 計算所有點到目標點的距離
    distances = [haversine(p, target_point, unit=Unit.METERS) for p in gpx_points]

    if not distances:
        return -1

    min_dist_idx_local = np.argmin(distances)
    closest_idx = start_search_idx + min_dist_idx_local

    return closest_idx


def calculate_segment_features(segment_df, all_comm_points):
    """
    計算單一路段的所有特徵。
    """
    features = {}

    if len(segment_df) < 2:
        return None  # 無法分析少於兩個點的路段

    segment_df = segment_df.copy()

    # 計算每一步的距離、海拔變化和時間差
    segment_df["distance_diff"] = [
        haversine(
            (row1["latitude"], row1["longitude"]),
            (row2["latitude"], row2["longitude"]),
            unit=Unit.METERS,
        )
        for row1, row2 in zip(
            segment_df.iloc[:-1].to_dict("records"),
            segment_df.iloc[1:].to_dict("records"),
        )
    ] + [0]

    segment_df["elevation_diff"] = segment_df["elevation"].diff().fillna(0)
    segment_df["time_diff"] = segment_df["time"].diff().dt.total_seconds().fillna(0)

    # 避免除以零
    segment_df["slope"] = np.degrees(
        np.arctan2(segment_df["elevation_diff"], segment_df["distance_diff"])
    )
    # 過濾掉因距離為零而產生的無效坡度
    valid_slopes = segment_df[segment_df["distance_diff"] > 0]["slope"]

    # 1. 距離 (distance)
    features["distance"] = segment_df["distance_diff"].sum()

    # 2. 海拔變化 (elevation_change) - 總爬升與總下降
    features["total_ascent"] = segment_df[segment_df["elevation_diff"] > 0][
        "elevation_diff"
    ].sum()
    features["total_descent"] = abs(
        segment_df[segment_df["elevation_diff"] < 0]["elevation_diff"].sum()
    )

    # 3. 坡度標準差 (slope_std_dev)
    features["slope_std_dev"] = valid_slopes.std() if not valid_slopes.empty else 0

    # 4. 坡度變異數 (slope_variance)
    features["slope_variance"] = valid_slopes.var() if not valid_slopes.empty else 0

    # 5. 最大坡度 (max_slope)
    features["max_uphill_slope"] = (
        valid_slopes[valid_slopes > 0].max()
        if not valid_slopes[valid_slopes > 0].empty
        else 0
    )
    features["max_downhill_slope"] = (
        valid_slopes[valid_slopes < 0].min()
        if not valid_slopes[valid_slopes < 0].empty
        else 0
    )

    # 6. 坡度頻率分布 (slope_freq_dist)
    bins = [-90, -10, -5, 5, 10, 90]
    labels = [
        "陡降(<-10°)",
        "緩降(-10°~-5°)",
        "平坦(-5°~5°)",
        "緩升(5°~10°)",
        "陡升(>10°)",
    ]
    total_distance = features["distance"]
    if total_distance > 0:
        freq_dist = segment_df.groupby(
            pd.cut(segment_df["slope"], bins=bins, labels=labels, right=False)
        )["distance_diff"].sum()
        features["slope_freq_dist"] = (freq_dist / total_distance * 100).to_dict()
    else:
        features["slope_freq_dist"] = {label: 0 for label in labels}

    # 7. 最大坡度高低時間差 (max_slope_time_diff) - 此處指產生最大坡度的單點間移動時間
    max_slope_idx = segment_df["slope"].abs().idxmax()
    features["max_slope_time_diff"] = segment_df.loc[max_slope_idx, "time_diff"]

    # 8. 切點間時間差 (breakpoint_time_diff)
    total_time_seconds = (
        segment_df["time"].iloc[-1] - segment_df["time"].iloc[0]
    ).total_seconds()
    features["breakpoint_time_diff"] = timedelta(seconds=total_time_seconds)

    # 9. 海拔範圍 (elevation_range)
    features["elevation_range"] = (
        segment_df["elevation"].max() - segment_df["elevation"].min()
    )
    features["max_elevation"] = segment_df["elevation"].max()

    # 10. 是否超過2438公尺海拔 (high_elevation)
    features["high_elevation"] = segment_df["elevation"].max() > 2438

    # 11. 最大坡度點 (max_slope_point)
    if not pd.isna(max_slope_idx) and max_slope_idx > segment_df.index[0]:
        p1 = segment_df.loc[max_slope_idx - 1]
        p2 = segment_df.loc[max_slope_idx]
        features["max_slope_point"] = (
            f"從 ({p1['latitude']:.5f}, {p1['longitude']:.5f}) 到 ({p2['latitude']:.5f}, {p2['longitude']:.5f})"
        )
    else:
        features["max_slope_point"] = "無足夠數據"

    # 12. 通訊點數量 (comm_point_count)
    count = 0
    segment_start_idx = segment_df.index[0]
    segment_end_idx = segment_df.index[-1]
    for idx, comm_point in all_comm_points.iterrows():
        closest_gpx_idx = find_closest_point_index(segment_df, comm_point)
        # 檢查最近點是否在當前路段的索引範圍內
        if (
            closest_gpx_idx != -1
            and segment_start_idx
            <= (segment_df.index[0] + closest_gpx_idx)
            <= segment_end_idx
        ):
            count += 1
    features["comm_point_count"] = count

    # 自定義效率指標
    # 效率 = 水平時速(km/h) + (垂直爬升速度(m/h) / 100)
    horizontal_speed_kmh = (
        (features["distance"] / 1000) / (total_time_seconds / 3600)
        if total_time_seconds > 0
        else 0
    )
    vertical_speed_mh = (
        features["total_ascent"] / (total_time_seconds / 3600)
        if total_time_seconds > 0
        else 0
    )
    features["efficiency_score"] = horizontal_speed_kmh + (vertical_speed_mh / 100)

    return features


def generate_markdown_report(features, segment_num, gpx_filename):
    """
    根據計算出的特徵，生成 Markdown 格式的報告。
    """
    if features is None:
        return (
            f"## {gpx_filename} - 路段 {segment_num}\n\n無法分析此路段，軌跡點少於2個。"
        )

    # --- 定性觀察的描述性文字 ---
    time_str = str(features["breakpoint_time_diff"]).split(".")[0]
    distance_km = features["distance"] / 1000

    # 基本描述
    qualitative_obs = (
        f"此路段全長 **{distance_km:.2f} 公里**，總計花費 **{time_str}**。"
    )
    qualitative_obs += f" 包含 **{features['total_ascent']:.1f} 公尺** 的總爬升與 **{features['total_descent']:.1f} 公尺** 的總下降。\n"

    # 坡度描述
    if features["slope_std_dev"] > 10:
        slope_consistency = "非常劇烈，充滿了陡峭的上下坡"
    elif features["slope_std_dev"] > 5:
        slope_consistency = "有一定起伏"
    else:
        slope_consistency = "相對平緩"
    qualitative_obs += f"路段的坡度變化 **{slope_consistency}**，最陡的上坡達到 **{features['max_uphill_slope']:.1f}°**，最陡的下坡則為 **{features['max_downhill_slope']:.1f}°**。\n"

    # 坡度分布描述
    flat_perc = features["slope_freq_dist"].get("平坦(-5°~5°)", 0)
    uphill_perc = features["slope_freq_dist"].get("陡升(>10°)", 0)
    downhill_perc = features["slope_freq_dist"].get("陡降(<-10°)", 0)
    qualitative_obs += (
        f"從坡度分佈來看，大約 **{flat_perc:.1f}%** 的路段為平坦地形，"
        f"而 **{uphill_perc:.1f}%** 為陡峭上坡，**{downhill_perc:.1f}%** 為陡峭下坡。"
        "這顯示出此路段的主要挑戰在於應對 **"
        + ("爬升" if uphill_perc > downhill_perc else "下降")
        + "**。\n"
    )

    # 海拔描述
    if features["high_elevation"]:
        qualitative_obs += f"**注意：此路段已進入高海拔區域** (超過 2438 公尺)，最高點達到 **{features['max_elevation']:.0f} 公尺**，需留意高山症風險。\n"
    else:
        qualitative_obs += f"此路段最高海拔為 **{features['max_elevation']:.0f} 公尺**，未進入高山症好發的高度區域。\n"

    # 通訊點描述
    qualitative_obs += f"路段範圍內共經過 **{features['comm_point_count']}** 個通訊點，提供了基本的通訊保障。\n"

    # 效率描述
    if features["efficiency_score"] > 4:
        efficiency_class = "高正效率"
        efficiency_desc = (
            "這通常代表著在爬升路段維持了良好的速度，或是平路/下坡時移動非常迅速。"
        )
    elif features["efficiency_score"] > 2.5:
        efficiency_class = "中等效率"
        efficiency_desc = "移動速度穩定，體力分配良好。"
    else:
        efficiency_class = "高負效率 (或低移動效率)"
        efficiency_desc = "這可能表示此路段極為陡峭、路況艱難，導致移動緩慢；也可能是停留休息時間較長所致。"

    # --- 報告內容組合 ---
    report = f"""
# GPX 分析報告: {gpx_filename} - 路段 {segment_num}

## 1. 統計數據摘要

| 特徵名稱 | 說明 | 數值 |
| :--- | :--- | :--- |
| **距離** | 水平線段長度 | `{distance_km:.2f} 公里` |
| **總爬升** | 該路段累積的爬升高度 | `+{features['total_ascent']:.1f} 公尺` |
| **總下降** | 該路段累積的下降高度 | `-{features['total_descent']:.1f} 公尺` |
| **切點間時間差** | 移動所花時間 | `{time_str}` |
| **海拔範圍** | 路段中最高與最低點之差 | `{features['elevation_range']:.1f} 公尺 (最高點: {features['max_elevation']:.0f} m)` |
| **是否超過2438公尺** | 是否有高山症風險 | `{'是' if features['high_elevation'] else '否'}` |
| **坡度標準差** | 坡度波動程度 | `{features['slope_std_dev']:.2f}°` |
| **坡度變異數** | 坡度變動整體幅度 | `{features['slope_variance']:.2f}` |
| **最大上坡/下坡** | 最極端的瞬間坡度 | `{features['max_uphill_slope']:.1f}° / {features['max_downhill_slope']:.1f}°` |
| **最大坡度點** | 發生最大坡度的軌跡點 | `{features['max_slope_point']}` |
| **最大坡度點時間差** | 產生最大坡度兩點間的時間 | `{features['max_slope_time_diff']:.1f} 秒` |
| **通訊點數量** | 該路段內的通訊點數量 | `{features['comm_point_count']} 個` |
| **坡度頻率分布** | 各坡度區間的距離佔比 | `{', '.join([f'{k}:{v:.1f}%' for k, v in features['slope_freq_dist'].items()])}` |

---

## 2. 定性觀察與詮釋

{qualitative_obs}

---

## 3. 指標定義與分類說明

### 效率指標 (Efficiency Score)

- **定義**: 這是一個綜合性的指標，用於評估在特定路段的移動效率。它的計算方式為：`效率分數 = 水平時速(km/h) + (垂直爬升速度(m/h) / 100)`。這個公式給予爬升速度較高的權重，因為在登山活動中，維持爬升速度是體能與技巧的關鍵展現。
- **數據差異說明**: 效率分數的範圍可能很廣，因為它受到地形、路況、個人體能、休息頻率等多重因素影響。陡峭或困難的路段分數自然較低，而平緩的下坡路段分數會較高。

### 本路段效率分類: **{efficiency_class}** (分數: {features['efficiency_score']:.2f})

- **詮釋**: {efficiency_desc}

"""
    return report


def main():
    """
    主執行函式。
    """
    print("--- GPX 分析腳本已啟動 ---")

    # 檢查 GPX 檔案是否存在
    if not os.path.exists(GPX_FILE_PATH):
        print(f"錯誤：GPX 檔案 '{GPX_FILE_PATH}' 不存在。請檢查檔案路徑。")
        return

    # 1. 載入資料
    print("正在載入通訊點資料...")
    comm_points_df = load_comm_points(COMMUNICATION_POINTS_DATA)
    breakpoints_df = comm_points_df[comm_points_df["是否為切點"] == "Y"].reset_index(
        drop=True
    )

    print(f"正在解析 GPX 檔案: {GPX_FILE_PATH}...")
    gpx_df = parse_gpx(GPX_FILE_PATH)
    print(f"GPX 檔案解析完成，共 {len(gpx_df)} 個軌跡點。")

    # 2. 尋找切點在 GPX 軌跡中的對應索引
    print("正在定位路線切點...")
    segment_indices = [0]
    last_found_gpx_index = 0

    # 尋找去程切點
    for idx, bp in breakpoints_df.iterrows():
        print(f"  - 尋找去程切點: {bp['路標指示']}...")
        closest_idx = find_closest_point_index(
            gpx_df, bp, start_search_idx=last_found_gpx_index
        )
        if closest_idx != -1 and closest_idx not in segment_indices:
            segment_indices.append(closest_idx)
            last_found_gpx_index = closest_idx
        else:
            print(
                f"    警告：無法為 {bp['路標指示']} 找到唯一的去程點，或與前一點重複。"
            )

    # 尋找返程切點 (從倒數第二個切點開始反向尋找)
    for idx, bp in breakpoints_df.iloc[:-1].iloc[::-1].iterrows():
        print(f"  - 尋找返程切點: {bp['路標指示']}...")
        closest_idx = find_closest_point_index(
            gpx_df, bp, start_search_idx=last_found_gpx_index + 1
        )  # 從上一個點之後開始找
        if closest_idx != -1 and closest_idx not in segment_indices:
            segment_indices.append(closest_idx)
            last_found_gpx_index = closest_idx
        else:
            print(
                f"    警告：無法為 {bp['路標指示']} 找到唯一的返程點，或與前一點重複。"
            )

    # 加入終點
    segment_indices.append(len(gpx_df) - 1)

    # 去除因距離太近而產生的重複索引
    segment_indices = sorted(list(set(segment_indices)))

    print(f"路線切點定位完成。共識別出 {len(segment_indices) - 1} 個路段。")
    print(f"切點索引: {segment_indices}")

    # 3. 分析每個路段並生成報告
    gpx_filename_base = os.path.basename(GPX_FILE_PATH).replace(".gpx", "")
    for i in range(len(segment_indices) - 1):
        start_idx = segment_indices[i]
        end_idx = segment_indices[i + 1]

        # 如果起點和終點索引相同，則跳過
        if start_idx >= end_idx:
            print(
                f"跳過路段 {i+1}，因為起點({start_idx})與終點({end_idx})索引相同或無效。"
            )
            continue

        print(f"\n--- 正在分析路段 {i+1} (軌跡點 {start_idx} 到 {end_idx}) ---")
        segment_df = gpx_df.iloc[start_idx : end_idx + 1]

        features = calculate_segment_features(segment_df, comm_points_df)

        report_content = generate_markdown_report(features, i + 1, gpx_filename_base)

        report_filename = f"{gpx_filename_base}-路段{i+1}.md"
        with open(report_filename, "w", encoding="utf-8") as f:
            f.write(report_content)

        print(f"路段 {i+1} 分析完成，報告已儲存至: {report_filename}")

    print("\n--- 所有分析已完成！ ---")


if __name__ == "__main__":
    main()

--- GPX 分析腳本已啟動 ---
正在載入通訊點資料...
正在解析 GPX 檔案: 「桃山」單攻.gpx...
GPX 檔案解析完成，共 9673 個軌跡點。
正在定位路線切點...
  - 尋找去程切點: 0 k 起點...
  - 尋找去程切點: 0K_瀑布1K交會...
    警告：無法為 0K_瀑布1K交會 找到唯一的去程點，或與前一點重複。
  - 尋找去程切點: 2 K...
    警告：無法為 2 K 找到唯一的去程點，或與前一點重複。
  - 尋找去程切點: 4.5 K...
    警告：無法為 4.5 K 找到唯一的去程點，或與前一點重複。
  - 尋找返程切點: 2 K...
  - 尋找返程切點: 0K_瀑布1K交會...
  - 尋找返程切點: 0 k 起點...
路線切點定位完成。共識別出 5 個路段。
切點索引: [0, np.int64(9661), np.int64(9662), np.int64(9663), np.int64(9664), 9672]

--- 正在分析路段 1 (軌跡點 0 到 9661) ---


C:\Users\pan\AppData\Local\Temp\ipykernel_9112\4172776980.py:129: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_dist = segment_df.groupby(pd.cut(segment_df['slope'], bins=bins, labels=labels, right=False))['distance_diff'].sum()
C:\Users\pan\AppData\Local\Temp\ipykernel_9112\4172776980.py:129: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_dist = segment_df.groupby(pd.cut(segment_df['slope'], bins=bins, labels=labels, right=False))['distance_diff'].sum()


路段 1 分析完成，報告已儲存至: 「桃山」單攻-路段1.md

--- 正在分析路段 2 (軌跡點 9661 到 9662) ---
路段 2 分析完成，報告已儲存至: 「桃山」單攻-路段2.md

--- 正在分析路段 3 (軌跡點 9662 到 9663) ---
路段 3 分析完成，報告已儲存至: 「桃山」單攻-路段3.md

--- 正在分析路段 4 (軌跡點 9663 到 9664) ---
路段 4 分析完成，報告已儲存至: 「桃山」單攻-路段4.md

--- 正在分析路段 5 (軌跡點 9664 到 9672) ---
路段 5 分析完成，報告已儲存至: 「桃山」單攻-路段5.md

--- 所有分析已完成！ ---


C:\Users\pan\AppData\Local\Temp\ipykernel_9112\4172776980.py:129: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_dist = segment_df.groupby(pd.cut(segment_df['slope'], bins=bins, labels=labels, right=False))['distance_diff'].sum()
C:\Users\pan\AppData\Local\Temp\ipykernel_9112\4172776980.py:129: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_dist = segment_df.groupby(pd.cut(segment_df['slope'], bins=bins, labels=labels, right=False))['distance_diff'].sum()
C:\Users\pan\AppData\Local\Temp\ipykernel_9112\4172776980.py:129: FutureWarning: The default of observed=False is deprecated and will be changed to 

In [ ]:
import gpxpy
import gpxpy.gpx
import pandas as pd
import numpy as np
from haversine import haversine, Unit
from datetime import timedelta
import io
import os

# --- 使用者設定 ---
# 將您的 GPX 檔案路徑放在這裡
GPX_FILE_PATH = "標準桃山.gpx"

# 通訊點資料檔案路徑
COMM_FILE_PATH = "communication_point.txt"

# --- 核心功能函式 ---


def load_comm_points(data):
    """
    從字串資料中讀取通訊點，並整理成 DataFrame。
    """
    df = pd.read_csv(io.StringIO(data), sep="\t")
    # 清理緯度和經度欄位中的特殊字元
    df["緯度"] = df["緯度"].str.replace("°", "").astype(float)
    df["經度"] = df["經度"].str.replace("°", "").astype(float)
    return df


def parse_gpx(file_path):
    """
    解析 GPX 檔案，將軌跡點轉換為 DataFrame。
    """
    points = []
    with open(file_path, "r", encoding="utf-8") as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    points.append(
                        {
                            "time": point.time,
                            "latitude": point.latitude,
                            "longitude": point.longitude,
                            "elevation": point.elevation,
                        }
                    )
    df = pd.DataFrame(points)
    # 將時間轉換為可操作的格式
    df["time"] = pd.to_datetime(df["time"], utc=True)
    return df


def find_closest_point_index(gpx_df, point, start_search_idx=0):
    """
    在 GPX DataFrame 中，從指定的索引開始，尋找距離給定點最近的點的索引。
    """
    min_dist = float("inf")
    closest_idx = -1

    # 建立一個 NumPy 陣列以進行向量化計算
    gpx_points = gpx_df[["latitude", "longitude"]].iloc[start_search_idx:].to_numpy()
    target_point = (point["緯度"], point["經度"])

    # 計算所有點到目標點的距離
    distances = [haversine(p, target_point, unit=Unit.METERS) for p in gpx_points]

    if not distances:
        return -1

    min_dist_idx_local = np.argmin(distances)
    closest_idx = start_search_idx + min_dist_idx_local

    return closest_idx


def calculate_segment_features(segment_df, all_comm_points):
    """
    計算單一路段的所有特徵。
    """
    features = {}

    if len(segment_df) < 2:
        return None  # 無法分析少於兩個點的路段

    segment_df = segment_df.copy()

    # 計算每一步的距離、海拔變化和時間差
    segment_df["distance_diff"] = [
        haversine(
            (row1["latitude"], row1["longitude"]),
            (row2["latitude"], row2["longitude"]),
            unit=Unit.METERS,
        )
        for row1, row2 in zip(
            segment_df.iloc[:-1].to_dict("records"),
            segment_df.iloc[1:].to_dict("records"),
        )
    ] + [0]

    segment_df["elevation_diff"] = segment_df["elevation"].diff().fillna(0)
    segment_df["time_diff"] = segment_df["time"].diff().dt.total_seconds().fillna(0)

    # 避免除以零
    segment_df["slope"] = np.degrees(
        np.arctan2(segment_df["elevation_diff"], segment_df["distance_diff"])
    )
    # 過濾掉因距離為零而產生的無效坡度
    valid_slopes = segment_df[segment_df["distance_diff"] > 0]["slope"]

    # 1. 距離 (distance)
    features["distance"] = segment_df["distance_diff"].sum()

    # 2. 海拔變化 (elevation_change) - 總爬升與總下降
    features["total_ascent"] = segment_df[segment_df["elevation_diff"] > 0][
        "elevation_diff"
    ].sum()
    features["total_descent"] = abs(
        segment_df[segment_df["elevation_diff"] < 0]["elevation_diff"].sum()
    )

    # 3. 坡度標準差 (slope_std_dev)
    features["slope_std_dev"] = valid_slopes.std() if not valid_slopes.empty else 0

    # 4. 坡度變異數 (slope_variance)
    features["slope_variance"] = valid_slopes.var() if not valid_slopes.empty else 0

    # 5. 最大坡度 (max_slope)
    features["max_uphill_slope"] = (
        valid_slopes[valid_slopes > 0].max()
        if not valid_slopes[valid_slopes > 0].empty
        else 0
    )
    features["max_downhill_slope"] = (
        valid_slopes[valid_slopes < 0].min()
        if not valid_slopes[valid_slopes < 0].empty
        else 0
    )

    # 6. 坡度頻率分布 (slope_freq_dist)
    bins = [-90, -10, -5, 5, 10, 90]
    labels = [
        "陡降(<-10°)",
        "緩降(-10°~-5°)",
        "平坦(-5°~5°)",
        "緩升(5°~10°)",
        "陡升(>10°)",
    ]
    total_distance = features["distance"]
    if total_distance > 0:
        freq_dist = segment_df.groupby(
            pd.cut(segment_df["slope"], bins=bins, labels=labels, right=False)
        )["distance_diff"].sum()
        features["slope_freq_dist"] = (freq_dist / total_distance * 100).to_dict()
    else:
        features["slope_freq_dist"] = {label: 0 for label in labels}

    # 7. 最大坡度高低時間差 (max_slope_time_diff) - 此處指產生最大坡度的單點間移動時間
    max_slope_idx = segment_df["slope"].abs().idxmax()
    features["max_slope_time_diff"] = segment_df.loc[max_slope_idx, "time_diff"]

    # 8. 切點間時間差 (breakpoint_time_diff)
    total_time_seconds = (
        segment_df["time"].iloc[-1] - segment_df["time"].iloc[0]
    ).total_seconds()
    features["breakpoint_time_diff"] = timedelta(seconds=total_time_seconds)

    # 9. 海拔範圍 (elevation_range)
    features["elevation_range"] = (
        segment_df["elevation"].max() - segment_df["elevation"].min()
    )
    features["max_elevation"] = segment_df["elevation"].max()

    # 10. 是否超過2438公尺海拔 (high_elevation)
    features["high_elevation"] = segment_df["elevation"].max() > 2438

    # 11. 最大坡度點 (max_slope_point)
    if not pd.isna(max_slope_idx) and max_slope_idx > segment_df.index[0]:
        p1 = segment_df.loc[max_slope_idx - 1]
        p2 = segment_df.loc[max_slope_idx]
        features["max_slope_point"] = (
            f"從 ({p1['latitude']:.5f}, {p1['longitude']:.5f}) 到 ({p2['latitude']:.5f}, {p2['longitude']:.5f})"
        )
    else:
        features["max_slope_point"] = "無足夠數據"

    # 12. 通訊點數量 (comm_point_count)
    count = 0
    segment_start_idx = segment_df.index[0]
    segment_end_idx = segment_df.index[-1]
    for idx, comm_point in all_comm_points.iterrows():
        closest_gpx_idx = find_closest_point_index(segment_df, comm_point)
        # 檢查最近點是否在當前路段的索引範圍內
        if (
            closest_gpx_idx != -1
            and segment_start_idx
            <= (segment_df.index[0] + closest_gpx_idx)
            <= segment_end_idx
        ):
            count += 1
    features["comm_point_count"] = count

    # 自定義效率指標
    # 效率 = 水平時速(km/h) + (垂直爬升速度(m/h) / 100)
    horizontal_speed_kmh = (
        (features["distance"] / 1000) / (total_time_seconds / 3600)
        if total_time_seconds > 0
        else 0
    )
    vertical_speed_mh = (
        features["total_ascent"] / (total_time_seconds / 3600)
        if total_time_seconds > 0
        else 0
    )
    features["efficiency_score"] = horizontal_speed_kmh + (vertical_speed_mh / 100)

    return features


def generate_markdown_report(features, segment_num, gpx_filename):
    """
    根據計算出的特徵，生成 Markdown 格式的報告。
    """
    if features is None:
        return (
            f"## {gpx_filename} - 路段 {segment_num}\n\n無法分析此路段，軌跡點少於2個。"
        )

    # --- 定性觀察的描述性文字 ---
    time_str = str(features["breakpoint_time_diff"]).split(".")[0]
    distance_km = features["distance"] / 1000

    # 基本描述
    qualitative_obs = (
        f"此路段全長 **{distance_km:.2f} 公里**，總計花費 **{time_str}**。"
    )
    qualitative_obs += f" 包含 **{features['total_ascent']:.1f} 公尺** 的總爬升與 **{features['total_descent']:.1f} 公尺** 的總下降。\n"

    # 坡度描述
    if features["slope_std_dev"] > 10:
        slope_consistency = "非常劇烈，充滿了陡峭的上下坡"
    elif features["slope_std_dev"] > 5:
        slope_consistency = "有一定起伏"
    else:
        slope_consistency = "相對平緩"
    qualitative_obs += f"路段的坡度變化 **{slope_consistency}**，最陡的上坡達到 **{features['max_uphill_slope']:.1f}°**，最陡的下坡則為 **{features['max_downhill_slope']:.1f}°**。\n"

    # 坡度分布描述
    flat_perc = features["slope_freq_dist"].get("平坦(-5°~5°)", 0)
    uphill_perc = features["slope_freq_dist"].get("陡升(>10°)", 0)
    downhill_perc = features["slope_freq_dist"].get("陡降(<-10°)", 0)
    qualitative_obs += (
        f"從坡度分佈來看，大約 **{flat_perc:.1f}%** 的路段為平坦地形，"
        f"而 **{uphill_perc:.1f}%** 為陡峭上坡，**{downhill_perc:.1f}%** 為陡峭下坡。"
        "這顯示出此路段的主要挑戰在於應對 **"
        + ("爬升" if uphill_perc > downhill_perc else "下降")
        + "**。\n"
    )

    # 海拔描述
    if features["high_elevation"]:
        qualitative_obs += f"**注意：此路段已進入高海拔區域** (超過 2438 公尺)，最高點達到 **{features['max_elevation']:.0f} 公尺**，需留意高山症風險。\n"
    else:
        qualitative_obs += f"此路段最高海拔為 **{features['max_elevation']:.0f} 公尺**，未進入高山症好發的高度區域。\n"

    # 通訊點描述
    qualitative_obs += f"路段範圍內共經過 **{features['comm_point_count']}** 個通訊點，提供了基本的通訊保障。\n"

    # 效率描述
    if features["efficiency_score"] > 4:
        efficiency_class = "高正效率"
        efficiency_desc = (
            "這通常代表著在爬升路段維持了良好的速度，或是平路/下坡時移動非常迅速。"
        )
    elif features["efficiency_score"] > 2.5:
        efficiency_class = "中等效率"
        efficiency_desc = "移動速度穩定，體力分配良好。"
    else:
        efficiency_class = "高負效率 (或低移動效率)"
        efficiency_desc = "這可能表示此路段極為陡峭、路況艱難，導致移動緩慢；也可能是停留休息時間較長所致。"

    # --- 報告內容組合 ---
    report = f"""
# GPX 分析報告: {gpx_filename} - 路段 {segment_num}

## 1. 統計數據摘要

| 特徵名稱 | 說明 | 數值 |
| :--- | :--- | :--- |
| **距離** | 水平線段長度 | `{distance_km:.2f} 公里` |
| **總爬升** | 該路段累積的爬升高度 | `+{features['total_ascent']:.1f} 公尺` |
| **總下降** | 該路段累積的下降高度 | `-{features['total_descent']:.1f} 公尺` |
| **切點間時間差** | 移動所花時間 | `{time_str}` |
| **海拔範圍** | 路段中最高與最低點之差 | `{features['elevation_range']:.1f} 公尺 (最高點: {features['max_elevation']:.0f} m)` |
| **是否超過2438公尺** | 是否有高山症風險 | `{'是' if features['high_elevation'] else '否'}` |
| **坡度標準差** | 坡度波動程度 | `{features['slope_std_dev']:.2f}°` |
| **坡度變異數** | 坡度變動整體幅度 | `{features['slope_variance']:.2f}` |
| **最大上坡/下坡** | 最極端的瞬間坡度 | `{features['max_uphill_slope']:.1f}° / {features['max_downhill_slope']:.1f}°` |
| **最大坡度點** | 發生最大坡度的軌跡點 | `{features['max_slope_point']}` |
| **最大坡度點時間差** | 產生最大坡度兩點間的時間 | `{features['max_slope_time_diff']:.1f} 秒` |
| **通訊點數量** | 該路段內的通訊點數量 | `{features['comm_point_count']} 個` |
| **坡度頻率分布** | 各坡度區間的距離佔比 | `{', '.join([f'{k}:{v:.1f}%' for k, v in features['slope_freq_dist'].items()])}` |

---

## 2. 定性觀察與詮釋

{qualitative_obs}

---

## 3. 指標定義與分類說明

### 效率指標 (Efficiency Score)

- **定義**: 這是一個綜合性的指標，用於評估在特定路段的移動效率。它的計算方式為：`效率分數 = 水平時速(km/h) + (垂直爬升速度(m/h) / 100)`。這個公式給予爬升速度較高的權重，因為在登山活動中，維持爬升速度是體能與技巧的關鍵展現。
- **數據差異說明**: 效率分數的範圍可能很廣，因為它受到地形、路況、個人體能、休息頻率等多重因素影響。陡峭或困難的路段分數自然較低，而平緩的下坡路段分數會較高。

### 本路段效率分類: **{efficiency_class}** (分數: {features['efficiency_score']:.2f})

- **詮釋**: {efficiency_desc}

"""
    return report


def main():
    """
    主執行函式。
    """
    print("--- GPX 分析腳本已啟動 ---")

    # 檢查 GPX 檔案是否存在
    if not os.path.exists(GPX_FILE_PATH):
        print(f"錯誤：GPX 檔案 '{GPX_FILE_PATH}' 不存在。請檢查檔案路徑。")
        return

    # 【修改點 2】檢查通訊點檔案是否存在
    if not os.path.exists(COMM_FILE_PATH):
        print(f"錯誤：通訊點檔案 '{COMM_FILE_PATH}' 不存在。請檢查檔案路徑。")
        return

    # 1. 載入資料
    print(f"正在從 '{COMM_FILE_PATH}' 載入通訊點資料...")
    # 【修改點 2】從外部檔案讀取通訊點資料
    with open(COMM_FILE_PATH, "r", encoding="utf-8") as f:
        comm_points_data_str = f.read()
    comm_points_df = load_comm_points(comm_points_data_str)
    breakpoints_df = comm_points_df[comm_points_df["是否為切點"] == "Y"].reset_index(
        drop=True
    )

    print(f"正在解析 GPX 檔案: {GPX_FILE_PATH}...")
    gpx_df = parse_gpx(GPX_FILE_PATH)
    print(f"GPX 檔案解析完成，共 {len(gpx_df)} 個軌跡點。")

    # 【修改點 1】從 GPX 檔名建立輸出資料夾
    gpx_filename_base = os.path.basename(GPX_FILE_PATH).replace(".gpx", "")
    output_dir = gpx_filename_base
    os.makedirs(output_dir, exist_ok=True)
    print(f"報告將儲存於資料夾: '{output_dir}/'")

    # 2. 尋找切點在 GPX 軌跡中的對應索引
    print("正在定位路線切點...")
    segment_indices = [0]
    last_found_gpx_index = 0

    # 尋找去程切點
    for idx, bp in breakpoints_df.iterrows():
        print(f"  - 尋找去程切點: {bp['路標指示']}...")
        closest_idx = find_closest_point_index(
            gpx_df, bp, start_search_idx=last_found_gpx_index
        )
        if closest_idx != -1 and closest_idx not in segment_indices:
            segment_indices.append(closest_idx)
            last_found_gpx_index = closest_idx
        else:
            print(
                f"    警告：無法為 {bp['路標指示']} 找到唯一的去程點，或與前一點重複。"
            )

    # 尋找返程切點 (從倒數第二個切點開始反向尋找)
    for idx, bp in breakpoints_df.iloc[:-1].iloc[::-1].iterrows():
        print(f"  - 尋找返程切點: {bp['路標指示']}...")
        closest_idx = find_closest_point_index(
            gpx_df, bp, start_search_idx=last_found_gpx_index + 1
        )  # 從上一個點之後開始找
        if closest_idx != -1 and closest_idx not in segment_indices:
            segment_indices.append(closest_idx)
            last_found_gpx_index = closest_idx
        else:
            print(
                f"    警告：無法為 {bp['路標指示']} 找到唯一的返程點，或與前一點重複。"
            )

    # 加入終點
    segment_indices.append(len(gpx_df) - 1)

    # 去除因距離太近而產生的重複索引
    segment_indices = sorted(list(set(segment_indices)))

    print(f"路線切點定位完成。共識別出 {len(segment_indices) - 1} 個路段。")
    print(f"切點索引: {segment_indices}")

    # 3. 分析每個路段並生成報告
    for i in range(len(segment_indices) - 1):
        start_idx = segment_indices[i]
        end_idx = segment_indices[i + 1]

        # 如果起點和終點索引相同，則跳過
        if start_idx >= end_idx:
            print(
                f"跳過路段 {i+1}，因為起點({start_idx})與終點({end_idx})索引相同或無效。"
            )
            continue

        print(f"\n--- 正在分析路段 {i+1} (軌跡點 {start_idx} 到 {end_idx}) ---")
        segment_df = gpx_df.iloc[start_idx : end_idx + 1]

        features = calculate_segment_features(segment_df, comm_points_df)

        report_content = generate_markdown_report(features, i + 1, gpx_filename_base)

        # 【修改點 1】設定報告檔名並儲存到指定資料夾
        report_filename = f"{gpx_filename_base}-路段{i+1}.md"
        output_path = os.path.join(output_dir, report_filename)

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(report_content)

        print(f"路段 {i+1} 分析完成，報告已儲存至: {output_path}")

    print("\n--- 所有分析已完成！ ---")


if __name__ == "__main__":
    main()

--- GPX 分析腳本已啟動 ---
正在從 'communication_point.txt' 載入通訊點資料...
正在解析 GPX 檔案: 標準桃山.gpx...
GPX 檔案解析完成，共 373 個軌跡點。
報告將儲存於資料夾: '標準桃山/'
正在定位路線切點...
  - 尋找去程切點: 0 k 起點...
  - 尋找去程切點: 0K_瀑布1K交會...
  - 尋找去程切點: 2 K...
  - 尋找去程切點: 4.5 K...
  - 尋找返程切點: 2 K...
  - 尋找返程切點: 0K_瀑布1K交會...
  - 尋找返程切點: 0 k 起點...
路線切點定位完成。共識別出 8 個路段。
切點索引: [0, np.int64(7), np.int64(49), np.int64(106), np.int64(161), np.int64(266), np.int64(322), np.int64(365), 372]

--- 正在分析路段 1 (軌跡點 0 到 7) ---
路段 1 分析完成，報告已儲存至: 標準桃山\標準桃山-路段1.md

--- 正在分析路段 2 (軌跡點 7 到 49) ---
路段 2 分析完成，報告已儲存至: 標準桃山\標準桃山-路段2.md

--- 正在分析路段 3 (軌跡點 49 到 106) ---
路段 3 分析完成，報告已儲存至: 標準桃山\標準桃山-路段3.md

--- 正在分析路段 4 (軌跡點 106 到 161) ---
路段 4 分析完成，報告已儲存至: 標準桃山\標準桃山-路段4.md

--- 正在分析路段 5 (軌跡點 161 到 266) ---
路段 5 分析完成，報告已儲存至: 標準桃山\標準桃山-路段5.md

--- 正在分析路段 6 (軌跡點 266 到 322) ---
路段 6 分析完成，報告已儲存至: 標準桃山\標準桃山-路段6.md

--- 正在分析路段 7 (軌跡點 322 到 365) ---
路段 7 分析完成，報告已儲存至: 標準桃山\標準桃山-路段7.md

--- 正在分析路段 8 (軌跡點 365 到 372) ---
路段 8 分析完成，報告已儲存至: 標準桃山\標準桃山-路段8.md

--- 所有分析已完成！ ---


C:\Users\student\AppData\Local\Temp\ipykernel_10244\1055402549.py:119: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_dist = segment_df.groupby(pd.cut(segment_df['slope'], bins=bins, labels=labels, right=False))['distance_diff'].sum()
C:\Users\student\AppData\Local\Temp\ipykernel_10244\1055402549.py:119: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  freq_dist = segment_df.groupby(pd.cut(segment_df['slope'], bins=bins, labels=labels, right=False))['distance_diff'].sum()
C:\Users\student\AppData\Local\Temp\ipykernel_10244\1055402549.py:119: FutureWarning: The default of observed=False is deprecated and will

In [ ]:
import markdown
from pathlib import Path


def generate_html_report(
    gpx_filename: str,
    segment_reports: list[str],
    all_features: list[dict],
    output_path: str,
):
    """
    將所有 Markdown 報告與總表彙整成 HTML 並寫入檔案。
    """
    # 將 Markdown 轉換為 HTML
    md = markdown.Markdown(extensions=["tables"])
    html_segments = [md.convert(report) for report in segment_reports]

    # 生成總表 HTML（彙整各段的關鍵數據）
    summary_rows = ""
    for i, feat in enumerate(all_features, start=1):
        summary_rows += f"""
        <tr>
            <td>路段 {i}</td>
            <td>{feat['distance'] / 1000:.2f} 公里</td>
            <td>+{feat['total_ascent']:.1f} / -{feat['total_descent']:.1f} 公尺</td>
            <td>{feat['breakpoint_time_diff']}</td>
            <td>{feat['efficiency_score']:.2f}</td>
            <td>{'是' if feat['high_elevation'] else '否'}</td>
            <td>{feat['comm_point_count']}</td>
        </tr>
        """

    summary_table = f"""
    <h2>總覽表：{gpx_filename}</h2>
    <table border="1" cellpadding="6" cellspacing="0">
        <tr>
            <th>路段</th>
            <th>距離</th>
            <th>爬升/下降</th>
            <th>時間</th>
            <th>效率指標</th>
            <th>高海拔</th>
            <th>通訊點數</th>
        </tr>
        {summary_rows}
    </table>
    """

    # 組成完整 HTML
    html_content = f"""
    <html>
    <head>
        <meta charset="utf-8">
        <title>{gpx_filename} 分析報告</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 2em; line-height: 1.6; }}
            table {{ border-collapse: collapse; margin-top: 1em; }}
            th, td {{ border: 1px solid #ccc; padding: 8px; text-align: center; }}
            h2 {{ color: #2c3e50; }}
        </style>
    </head>
    <body>
        <h1>GPX 分析總報告：{gpx_filename}</h1>
        {summary_table}
        <hr>
        {"<hr>".join(html_segments)}
    </body>
    </html>
    """

    # 寫入 HTML 檔案
    Path(output_path).write_text(html_content, encoding="utf-8")
    print(f"[完成] HTML 報告已輸出：{output_path}")